In [4]:
import pickle
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
from loguru import logger as log
import configparser
import uuid

In [5]:
config = configparser.ConfigParser()
config.read('../config.ini')

['../config.ini']

In [6]:
qdrant_client = QdrantClient(url=config['production']['QdrantUrl'])

qdrant_client.recreate_collection(
        collection_name=config['production']['CollectionName'],
        vectors_config=VectorParams(size=512, distance=Distance.COSINE),
    )

/var/folders/vg/w_s0_zp96vb5y35z_8hbwc200000gn/T/ipykernel_79789/3086519759.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [7]:
with open("../../data/data.pkl", "rb") as f:
    products = pickle.load(f)

In [8]:
for product in products:
    points = [
            PointStruct(
                id=uuid.uuid4().__str__(), 
                vector=embedding.tolist(),
                payload={**product[0]},
            )
            for embedding in product[1]
        ]
    
    qdrant_client.upsert(collection_name=config['production']['CollectionName'], points=points)
    log.info('embedding of product {} added to qdrant successfully',str(product[0]['id']))

2024-11-28 23:51:31.709 | INFO     | __main__:<module>:12 - embedding of product 2089163 added to qdrant successfully
2024-11-28 23:51:31.795 | INFO     | __main__:<module>:12 - embedding of product 2089164 added to qdrant successfully
2024-11-28 23:51:31.848 | INFO     | __main__:<module>:12 - embedding of product 2058607 added to qdrant successfully
2024-11-28 23:51:31.905 | INFO     | __main__:<module>:12 - embedding of product 2089165 added to qdrant successfully
2024-11-28 23:51:31.951 | INFO     | __main__:<module>:12 - embedding of product 2089166 added to qdrant successfully
2024-11-28 23:51:31.976 | INFO     | __main__:<module>:12 - embedding of product 2089167 added to qdrant successfully
2024-11-28 23:51:32.018 | INFO     | __main__:<module>:12 - embedding of product 2089168 added to qdrant successfully
2024-11-28 23:51:32.039 | INFO     | __main__:<module>:12 - embedding of product 2089169 added to qdrant successfully
2024-11-28 23:51:32.074 | INFO     | __main__:<module>:1